In [1]:
import json
import time
import pickle
import sys
import os

import zstandard as zstd
import pandas as pd
import numpy as np

from scipy.sparse import dok_matrix
#from helpers import *

In [ ]:
# Dictionnary mapping the video_id to the channel_id
vid_to_channels = pd.read_pickle("/dlabdata1/youtube_large/id_to_channel_mapping.pickle")

In [ ]:
# Channels that are in set_crawler dataset and also in which the language is in english
with open('/dlabdata1/youtube_large/olam/channels_id_filtered.pickle', 'rb') as f:
    channels_id = pickle.load(f)
f.close()
# Dictionnary to map the channel id to an integer corresponding to the column/row of the sparse matrix.
channel_dict = {}
channels_id = sorted(channels_id)
for ind, channel_id in enumerate(channels_id):
    channel_dict[channel_id] = ind
channels_id = set(channels_id)

#### Create the bipartite graph between the users and the channels using sparse matrices

In [ ]:
'''
Using Zreader enables to read the .zst files line by line
'''
class Zreader:

    def __init__(self, file, chunk_size=16384):
        '''Init method'''
        self.fh = open(file,'rb')
        self.chunk_size = chunk_size
        self.dctx = zstd.ZstdDecompressor()
        self.reader = self.dctx.stream_reader(self.fh)
        self.buffer = ''

    def readlines(self):
        '''Generator method that creates an iterator for each line of JSON'''
        while True:
            chunk = self.reader.read(self.chunk_size).decode("utf-8", errors="ignore")
            if not chunk:
                break
            lines = (self.buffer + chunk).split("\n")

            for line in lines[:-1]:
                yield line

            self.buffer = lines[-1]


In [ ]:
# Adjust chunk_size as necessary -- defaults to 16,384 if not specific
reader = Zreader("/dlabdata1/youtube_large/youtube_comments.ndjson.zst", chunk_size=16384)

# parameters
channel_to_nb_users_commented = {}
idx = 1
user_idx = 0
user = ''
begin_time = time.time()
nb_users = 0

#dir_1 = '/dlabdata1/youtube_large/jouven/channel_embedding/bipartite_sparse_matrix/'
#check_directory(dir_1)

# Read each line from the reader
for line in reader.readlines():
    line_split = line.replace('"', '').split(',')
    
    if idx > 1:
        if len(line_split) == 9:
            if vid_to_channels.get(line_split[2]) in channels_id:
                corr_channel = channel_dict[vid_to_channels[line_split[2]]]
                if corr_channel in channel_to_nb_users_commented:
                    channel_to_nb_users_commented[corr_channel] += 1
                else:
                    channel_to_nb_users_commented[corr_channel] = 1
            if line_split[0] != user:
                nb_users += 1
    user = line_split[0]
    
                
    idx += 1
    if idx % 1000000000 == 0:
        with open("/dlabdata1/youtube_large/jouven/idx_run.pkl",'wb') as f:
             pickle.dump([idx], f)
        f.close()
        
with open("/dlabdata1/youtube_large/jouven/channels_idx_to_nb_comments.pkl",'wb') as f:
     pickle.dump(channel_to_nb_users_commented, f)
f.close()
with open("/dlabdata1/youtube_large/jouven/nb_users.pkl",'wb') as f:
     pickle.dump([nb_users], f)
f.close()

In [18]:
len(channel_to_nb_users_commented)

52194

In [ ]:
def check_directory(dir_1):
    
    if not os.path.exists(dir_1): 
        os.makedirs(dir_1)

In [ ]:
def create_edges(graph_dict, user, user_channels):
    

In [13]:
# Adjust chunk_size as necessary -- defaults to 16,384 if not specific
reader = Zreader("/dlabdata1/youtube_large/youtube_comments.ndjson.zst", chunk_size=16384)

# parameters
max_freq = 0


graph_dict = {}
user_channels = []
idx = 1
user = ''
begin_time = time.time()

dir_1 = '/dlabdata1/youtube_large/jouven/channel_embedding/bipartite_sparse_matrix/'
#check_directory(dir_1)

# Read each line from the reader
for line in reader.readlines():
    line_split = line.replace('"', '').split(',')
    if len(line_split) == 9:
        if vid_to_channels.get(line_split[2]) in channels_id:
            corr_channel = vid_to_channels[line_split[2]]
            if line_split[0] == user:
                #user_channels.append(channel_dict[corr_channel])
                graph_dict[line_split[0]] = idx
                if len(graph_dict) == 1000:
                    print('Size of graph dict: ' + str(sys.getsizeof(graph_dict)))
                    break
            else:
                graph_dict[line_split[0]] = idx
                if len(graph_dict) == 1000:
                    print('Size of graph dict: ' + str(sys.getsizeof(graph_dict)))
                    break
                #create_edges(graph_dict_limited, user_channels)
                #if len(graph_dict) >= 100000000:
                    # For space requirements every 100 millions line create a dok matrix and
                    # update it with the graph_dict dictionnary and then save it into csr format and then release memory
                    #graph_matrix = dok_matrix((matrix_len, matrix_len), dtype=np.uint8)
                    #dict.update(graph_matrix, graph_dict)
                    #print('Size of matrix limited dok: ' + str(sys.getsizeof(graph_matrix)))
                    #graph_dict_limited = {}
                    # Save sparse matrix
                    #scipy.sparse.save_npz('../../../dlabdata1/youtube_large/jouven/sparse_matrices_limited/matrice' + str(nb) + '.npz', graph_matrix.tocsr())

                    #graph_matrix = []
                    #nb += 1
                    #print('line number: ' + str(idx) + ' time: ' + str(time.time() - begin_time))
                    #begin_time = time.time()
                
                user_channels = []
                user_channels.append(channel_dict[corr_channel])
                
    user = line_split[0]
    idx += 1
    

#graph_matrix = dok_matrix((matrix_len, matrix_len), dtype=np.uint8)
#dict.update(graph_matrix, graph_dict_limited)
#print('Size of matrix limited dok: ' + str(sys.getsizeof(graph_matrix)))
#graph_dict_limited = {}
# Save sparse matrix
#scipy.sparse.save_npz('../../../dlabdata1/youtube_large/jouven/sparse_matrices_limited/matrice' + str(nb) + '.npz', graph_matrix.tocsc())

graph_matrix = []


Size of graph dict: 36976


In [ ]:
def create_edges_normalized(graph_dict, user_edge):
    
    nb_comments = len(user_edge)
    user_edge = list(set(user_edge))
    sorted_user_edge = sorted(user_edge)
    
    if len(user_edge) >= 2:
        for comb in itertools.combinations(sorted_user_edge, 2):

            user_edge = (comb[0], comb[1])
            if user_edge in graph_dict:
                graph_dict[user_edge] += 1/(math.log(nb_comments, 2))
            else:
                graph_dict[user_edge] = 1/(math.log(nb_comments, 2))

            
def create_edges_limited_normalized(graph_dict, user_edge):
    
    nb_comments = len(user_edge)
    user_edge = list(set(user_edge))
    if len(user_edge) > 100:
        user_edge = random.sample(user_edge, 100)
    
    if len(user_edge) >= 2:
        sorted_user_edge = sorted(user_edge)
        for comb in itertools.combinations(sorted_user_edge, 2):
            user_edge = (comb[0], comb[1])
            if user_edge in graph_dict:
                graph_dict[user_edge] += 1/(math.log(nb_comments, 2))
            else:
                graph_dict[user_edge] = 1/(math.log(nb_comments, 2))

In [ ]:
def check_directory(dir_1):
    
    if not os.path.exists(dir_1): 
        os.makedirs(dir_1)

In [ ]:
with open("/dlabdata1/youtube_large/jouven/occurent_users.pkl",'rb') as f:
    set_occurent_users = pickle.load(f)
f.close()

In [ ]:
dict_user_occurence = {}
for val in set_occurent_users:
    dict_user_occurence[val] = 0
set_occurent_users = set(set_occurent_users)

In [ ]:
# Adjust chunk_size as necessary -- defaults to 16,384 if not specific
reader = Zreader("/dlabdata1/youtube_large/youtube_comments.ndjson.zst", chunk_size=16384)

# parameters
graph_dict_limited = {}
nb = 1
idx = 1
user_channels = []
user_idx = -1
user_bool = True

user = ''
begin_time = time.time()

dir_1 = '/dlabdata1/youtube_large/jouven/sparse_matrix_construction/sparse_matrices_limited_normalized/'
check_directory(dir_1)

# Read each line from the reader
for line in reader.readlines():
    line_split = line.replace('"', '').split(',')
    if len(line_split) == 9:
        if vid_to_channels.get(line_split[2]) in channels_id:
            corr_channel = vid_to_channels[line_split[2]]
            if user_idx in dict_user_occurence:
                dict_user_occurence[user_idx] += 1
                if dict_user_occurence[user_idx] > 1:
                    user_bool = False
            else:
                user_bool = True
                
            if user_bool:
                if line_split[0] == user:
                    user_channels.append(channel_dict[corr_channel])
                else:
                    if len(user_channels) >= 2:
                        create_edges_limited_normalized(graph_dict_limited, user_channels)
                    #print(graph_dict_limited)
                    if len(graph_dict_limited) >= 100000000:
                        # For space requirements every 100 millions line create a dok matrix and
                        # update it with the graph_dict dictionnary and then save it into csr format and then release memory
                        graph_matrix = dok_matrix((matrix_len, matrix_len), dtype=np.uint8)
                        dict.update(graph_matrix, graph_dict_limited)
                        #print('Size of matrix limited dok: ' + str(sys.getsizeof(graph_matrix)))
                        print(graph_matrix.count_nonzero())
                        graph_dict_limited = {}
                        # Save sparse matrix
                        scipy.sparse.save_npz('/dlabdata1/youtube_large/jouven/sparse_matrix_construction/sparse_matrices_limited_normalized/matrice' + str(nb) + '.npz', graph_matrix.tocsr())
                        with open("/dlabdata1/youtube_large/jouven/sparse_matrix_construction/idx.pkl",'wb') as f:
                             pickle.dump([idx], f)
                        f.close()
                        graph_matrix = []
                        nb += 1
                        print('line number: ' + str(idx) + ' time: ' + str(time.time() - begin_time))
                        begin_time = time.time()

                    user_channels = []
                    user_channels.append(channel_dict[corr_channel])
                    user_idx += 1
            else:
                if line_split[0] != user:
                    user_idx += 1
    user = line_split[0]
    idx += 1
    

graph_matrix = dok_matrix((matrix_len, matrix_len), dtype=np.uint8)
dict.update(graph_matrix, graph_dict_limited)
print('Size of matrix limited dok: ' + str(sys.getsizeof(graph_matrix)))
graph_dict_limited = {}
# Save sparse matrix
scipy.sparse.save_npz('/dlabdata1/youtube_large/jouven/sparse_matrix_construction/sparse_matrices_limited_normalized/matrice' + str(nb) + '.npz', graph_matrix.tocsr())

graph_matrix = []
